In [85]:
!wget https://raw.githubusercontent.com/towardsai/tutorials/master/sentiment_analysis_tutorial/women_clothing_review.csv

--2021-01-29 20:10:39--  https://raw.githubusercontent.com/towardsai/tutorials/master/sentiment_analysis_tutorial/women_clothing_review.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8480236 (8.1M) [text/plain]
Saving to: ‘women_clothing_review.csv.10’

women_clothing_revi 100%[===================>]   8.09M  49.3MB/s    in 0.2s    

2021-01-29 20:10:40 (49.3 MB/s) - ‘women_clothing_review.csv.10’ saved [8480236/8480236]



In [86]:
import pandas as pd
import numpy as np
import string
from string import punctuation
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import tensorflow as tf

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [87]:
df = pd.read_csv('women_clothing_review.csv')



df.head(10)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
5,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
6,6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits
7,7,858,39,"Shimmer, surprisingly goes with lots","I ordered this in carbon for store pick up, an...",4,1,4,General Petite,Tops,Knits
8,8,1077,24,Flattering,I love this dress. i usually get an xs but it ...,5,1,0,General,Dresses,Dresses
9,9,1077,34,Such a fun dress!,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,1,0,General,Dresses,Dresses


In [88]:
df = df.drop(['Title', 'Positive Feedback Count', 'Unnamed: 0', ], axis=1)
df.dropna(inplace=True)
df['Polarity_Rating'] = df['Rating'].apply(lambda x: 'Positive' if x > 3 else('Neutral' if x == 3  else 'Negative'))
df.head()

,Clothing ID,Age,Review Text,Rating,Recommended IND,Division Name,Department Name,Class Name,Polarity_Rating
0,767,33,Absolutely wonderful - silky and sexy and comf...,4,1,Initmates,Intimate,Intimates,Positive
1,1080,34,Love this dress! it's sooo pretty. i happene...,5,1,General,Dresses,Dresses,Positive
2,1077,60,I had such high hopes for this dress and reall...,3,0,General,Dresses,Dresses,Neutral
3,1049,50,"I love, love, love this jumpsuit. it's fun, fl...",5,1,General Petite,Bottoms,Pants,Positive
4,847,47,This shirt is very flattering to all due to th...,5,1,General,Tops,Blouses,Positive


In [89]:
df_Positive = df[df['Polarity_Rating'] == 'Positive'].sample(8000, replace=True)
df_Neutral = df[df['Polarity_Rating'] == 'Neutral'].sample(8000, replace=True)
df_Negative = df[df['Polarity_Rating'] == 'Negative'].sample(8000, replace=True)


df = pd.concat([df_Positive, df_Neutral, df_Negative], axis=0)
df.head()

,Clothing ID,Age,Review Text,Rating,Recommended IND,Division Name,Department Name,Class Name,Polarity_Rating
10570,1110,39,Great job from the fabricators in creating suc...,5,1,General,Dresses,Dresses,Positive
11932,829,33,I had the same problem i usually have with but...,4,1,General Petite,Tops,Blouses,Positive
23235,227,62,I love this top. i'm wearing it today. i bough...,5,1,Initmates,Intimate,Layering,Positive
13376,936,38,The ivory color needs a little camisole or tan...,5,1,General Petite,Tops,Sweaters,Positive
16865,1104,42,I'm a 34c and 136 lbs. i bought this in m in t...,5,1,General,Dresses,Dresses,Positive


In [90]:
def preprocess_test(text):
    stpword = stopwords.words('english')
    no_punctuation = [char for char in text if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return ' '.join([word for word in no_punctuation.split() if word.lower() not in stpword])

df['review'] = df['Review Text'].apply(preprocess_test)
df = df[['review', 'Polarity_Rating']]
df.head()

,review,Polarity_Rating
10570,Great job fabricators creating gorgeous print ...,Positive
11932,problem usually buttonfront shirts tried gappi...,Positive
23235,love top im wearing today bought wear blazers ...,Positive
13376,ivory color needs little camisole tank underne...,Positive
16865,Im 34c 136 lbs bought two colors love much fit...,Positive


In [91]:
one_hot = pd.get_dummies(df["Polarity_Rating"])
df.drop(['Polarity_Rating'],axis=1,inplace=True)
df = pd.concat([df,one_hot],axis=1)
df.head()

,review,Negative,Neutral,Positive
10570,Great job fabricators creating gorgeous print ...,0,0,1
11932,problem usually buttonfront shirts tried gappi...,0,0,1
23235,love top im wearing today bought wear blazers ...,0,0,1
13376,ivory color needs little camisole tank underne...,0,0,1
16865,Im 34c 136 lbs bought two colors love much fit...,0,0,1


In [92]:
x = df['review'].values
y = df.drop('review', axis=1).values
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.30, random_state=42)
prediction_inputs = [xtest[:10], ytest[:10]]

In [93]:
vect = CountVectorizer()
xtrain = vect.fit_transform(xtrain)
xtest = vect.transform(xtest)

In [94]:
tfidf = TfidfTransformer()
xtrain = tfidf.fit_transform(xtrain)
xtest = tfidf.transform(xtest)
xtrain = xtrain.toarray()
xtest = xtest.toarray()

In [95]:
k = tf.keras
model = k.Sequential()

model.add(k.layers.Dense(units=1024,activation='relu'))
model.add(k.layers.Dropout(0.5))

model.add(k.layers.Dense(units=512,activation='relu'))
model.add(k.layers.Dropout(0.5))

model.add(k.layers.Dense(units=256,activation='relu'))
model.add(k.layers.Dropout(0.5))

model.add(k.layers.Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.fit(x=xtrain, y=ytrain, batch_size=256, epochs=1, validation_data=(xtest, ytest))

model_score = model.evaluate(xtest, ytest, batch_size=64, verbose=1)
print('Test accuracy:', model_score[1])

113/113 [==============================] - 4s 33ms/step - loss: 0.4872 - accuracy: 0.8079
Test accuracy: 0.8079166412353516


In [96]:
preds = model.predict(xtest)
for pred in preds[:10]:
  print(np.where(pred == pred.max())[0])
print(prediction_inputs)

[1]
[0]
[1]
[0]
[0]
[2]
[2]
[2]
[1]
[0]
[array(['colors beautiful wider hips thighs blouse looks boxy ordered size aspect fit however flowy boxy body type considered keeping layering even sale expensive purpose im bummed colors pretty',
       'fit strange middle string definitely long thin halter top offers hardly lift print lovely quality nice price cups padded nice suppose would barely cover anything bigger c cup im guessing im 32dd size 8 pants size 46 dresses ordered medium also long torso probably didnt help',
       'love design top however quality lackluster shirt pretty short would flattering least inches also pretty sheer figuring garments dilemma still wear love looks great denim vest',
       'looked fabulous model course love sweater dresses even xs drowning fabric looked ludicrous concept good execution went terribly wrong somewhere definite return',
       'rather boring tank thought itd perfect cardigan work decided fit boxy look good girl curves',
       'Bought navy s